**Imports**

In [45]:
'''
# Use the data in DLGN and its variations from https://arxiv.org/abs/2010.04627
'''
#Imports for Latent Tree data

import random
import requests
import os
import torch
from tqdm import tqdm
import numpy as np
import gzip
import shutil
import tarfile
import bz2
import pandas as pd
import gzip
import shutil
import warnings

from pathlib import Path
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_swiss_roll
from sklearn.preprocessing import QuantileTransformer

from category_encoders import LeaveOneOutEncoder
from category_encoders.ordinal import OrdinalEncoder
import os
import zipfile
import shutil
import urllib.request
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import NearestCentroid
from scipy.io import arff
# !pip install numpy==1.22.0  # Install a compatible version of NumPy
# !pip install scipy  # Install or update SciPy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import copy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import product as cartesian_prod
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import pairwise_distances

from sklearn import tree
from sklearn import cluster, mixture


np.set_printoptions(precision=4)


#@title Importing Packages
import os
import random
import pandas as pd
from copy import deepcopy
import torchvision
import torchvision.transforms as transforms

import time
import sys

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


**Download dataset**

In [46]:
#**from src.utils import download**
def download(url, filename, delete_if_interrupted=True, chunk_size=4096):
    """ saves file from url to filename with a fancy progressbar """
    try:
        with open(filename, "wb") as f:
            print("Downloading {} > {}".format(url, filename))
            response = requests.get(url, stream=True)
            total_length = response.headers.get('content-length')

            if total_length is None:  # no content length header
                f.write(response.content)
            else:
                total_length = int(total_length)
                with tqdm(total=total_length) as progressbar:
                    for data in response.iter_content(chunk_size=chunk_size):
                        if data:  # filter-out keep-alive chunks
                            f.write(data)
                            progressbar.update(len(data))

    except Exception as e:
        if delete_if_interrupted:
            print("Removing incomplete download {}.".format(filename))
            os.remove(filename)
        raise e
    return filename

#**from src.clus_datasets import ***
#**CLUSTERING DATASET FETCHERS**

# -------------------------------------------------------- CLUSTERING DATASET FETCHERS

def fetch_GLASS(path, valid_size=0.2, test_size=0.2, seed=None):

    path = Path(path)
    data_path = path / 'glass.data'

    if not data_path.exists():
        path.mkdir(parents=True)

        download('https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data', data_path)

    data = np.genfromtxt(data_path, delimiter=',')
    
    X, Y = (data[:, 1:-1]).astype(np.float32), (data[:, -1] - 1).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=test_size, random_state=seed)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=valid_size / (1 - test_size), random_state=seed)

    return dict(
        X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test
    )

def fetch_COVTYPE(path, valid_size=0.2, test_size=0.2, seed=None):

    path = Path(path)
    data_path = path / 'covtype.data'

    if not data_path.exists():
        path.mkdir(parents=True)
        archive_path = path / 'covtype.data.gz'
        download('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', archive_path)

        with gzip.open(archive_path, 'rb') as f_in:

            with open(data_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    data = np.genfromtxt(data_path, delimiter=',')
    
    X, Y = (data[:, :-1]).astype(np.float32), (data[:, -1] - 1).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=test_size, random_state=seed)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=valid_size / (1 - test_size), random_state=seed)

    return dict(
        X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test
    )

def fetch_ALOI(path, valid_size=0.2, test_size=0.2, seed=None):

    from PIL import Image
    from tqdm import tqdm

    path = Path(path)
    data_path = path / 'aloi_red4'
    npz_path = path / 'aloi_red4.npz'

    if not data_path.exists():

        path.mkdir(parents=True, exist_ok=True)

        for data_type in ["ill", "col", "view", "stereo"]:
            
            archive_path = path / f'aloi_red4_{data_type}.tar'

            download(f'http://aloi.science.uva.nl/tars/aloi_red4_{data_type}.tar', archive_path)

            with tarfile.open(archive_path, 'r') as f_in:

                f_in.extractall(path=data_path)

    if not npz_path.exists():
        
        X = np.empty((110250, 3, 144, 192), dtype=np.uint8)
        Y = np.empty(110250, dtype=np.uint16)

        # loop over classes
        i = 0
        for c in tqdm(range(1000), desc='Converting ALOI png to npy'):

            c_path = data_path / "png4" / str(c + 1) 

            # loop over class instances
            for i_path in c_path.glob('*.png'):
                
                im_frame = Image.open(i_path)
                X[i] = np.transpose(np.array(im_frame), (2, 0, 1))
                Y[i] = c
                i += 1

        np.savez_compressed(npz_path, X=X, Y=Y)
    
    else:

        data = np.load(npz_path)
        X, Y = data['X'], data['Y']

    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=test_size, random_state=seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=valid_size / (1 - test_size), random_state=seed)

    return dict(
        X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test
    )

def fetch_DIGITS(path, valid_size=0.2, test_size=0.2, seed=None):

    from sklearn.datasets import load_digits

    X, Y = load_digits(return_X_y=True)
    X, Y = X.reshape(-1, 1, 8, 8).astype(np.uint8), Y.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=test_size, random_state=seed)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=valid_size / (1 - test_size), random_state=seed)

    return dict(
        X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test
    )

#**from src.tabular_datasets import ***

def fetch_A9A(path, train_size=None, valid_size=None, test_size=None, **kwargs):
    train_path = os.path.join(path, 'a9a')
    test_path = os.path.join(path, 'a9a.t')
    if not all(os.path.exists(fname) for fname in (train_path, test_path)):
        os.makedirs(path, exist_ok=True)
        download("https://www.dropbox.com/s/9cqdx166iwonrj9/a9a?dl=1", train_path)
        download("https://www.dropbox.com/s/sa0ds895c0v4xc6/a9a.t?dl=1", test_path)

    X_train, y_train = load_svmlight_file(train_path, dtype=np.float32, n_features=123)
    X_test, y_test = load_svmlight_file(test_path, dtype=np.float32, n_features=123)
    X_train, X_test = X_train.toarray(), X_test.toarray()
    y_train[y_train == -1] = 0
    y_test[y_test == -1] = 0
    y_train, y_test = y_train.astype(np.int), y_test.astype(np.int)

    if all(sizes is None for sizes in (train_size, valid_size, test_size)):
        train_idx_path = os.path.join(path, 'stratified_train_idx.txt')
        valid_idx_path = os.path.join(path, 'stratified_valid_idx.txt')
        if not all(os.path.exists(fname) for fname in (train_idx_path, valid_idx_path)):
            download("https://www.dropbox.com/s/xy4wwvutwikmtha/stratified_train_idx.txt?dl=1", train_idx_path)
            download("https://www.dropbox.com/s/nthpxofymrais5s/stratified_test_idx.txt?dl=1", valid_idx_path)
        train_idx = pd.read_csv(train_idx_path, header=None)[0].values
        valid_idx = pd.read_csv(valid_idx_path, header=None)[0].values
    else:
        assert train_size, "please provide either train_size or none of sizes"
        if valid_size is None:
            valid_size = len(X_train) - train_size
            assert valid_size > 0
        if train_size + valid_size > len(X_train):
            warnings.warn('train_size + valid_size = {} exceeds dataset size: {}.'.format(
                train_size + valid_size, len(X_train)), Warning)
        if test_size is not None:
            warnings.warn('Test set is fixed for this dataset.', Warning)

        shuffled_indices = np.random.permutation(np.arange(len(X_train)))
        train_idx = shuffled_indices[:train_size]
        valid_idx = shuffled_indices[train_size: train_size + valid_size]    

    return dict(
        X_train=X_train[train_idx], y_train=y_train[train_idx],
        X_valid=X_train[valid_idx], y_valid=y_train[valid_idx],
        X_test=X_test, y_test=y_test
    )

def fetch_EPSILON(path, train_size=None, valid_size=None, test_size=None, **kwargs):
    train_path = os.path.join(path, 'epsilon_normalized')
    test_path = os.path.join(path, 'epsilon_normalized.t')
    if not all(os.path.exists(fname) for fname in (train_path, test_path)):
        os.makedirs(path, exist_ok=True)
        train_archive_path = os.path.join(path, 'epsilon_normalized.bz2')
        test_archive_path = os.path.join(path, 'epsilon_normalized.t.bz2')
        if not all(os.path.exists(fname) for fname in (train_archive_path, test_archive_path)):
            download("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/epsilon_normalized.bz2", train_archive_path)
            download("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/epsilon_normalized.t.bz2", test_archive_path)
        print("unpacking dataset")
        for file_name, archive_name in zip((train_path, test_path), (train_archive_path, test_archive_path)):
            zipfile = bz2.BZ2File(archive_name)
            with open(file_name, 'wb') as f:
                f.write(zipfile.read())

    print("reading dataset (it may take a long time)")
    X_train, y_train = load_svmlight_file(train_path, dtype=np.float32, n_features=2000)
    X_test, y_test = load_svmlight_file(test_path, dtype=np.float32, n_features=2000)
    X_train, X_test = X_train.toarray(), X_test.toarray()
    y_train, y_test = y_train.astype(np.int), y_test.astype(np.int)
    y_train[y_train == -1] = 0
    y_test[y_test == -1] = 0

    if all(sizes is None for sizes in (train_size, valid_size, test_size)):
        train_idx_path = os.path.join(path, 'stratified_train_idx.txt')
        valid_idx_path = os.path.join(path, 'stratified_valid_idx.txt')
        if not all(os.path.exists(fname) for fname in (train_idx_path, valid_idx_path)):
            download("https://www.dropbox.com/s/wxgm94gvm6d3xn5/stratified_train_idx.txt?dl=1", train_idx_path)
            download("https://www.dropbox.com/s/fm4llo5uucdglti/stratified_valid_idx.txt?dl=1", valid_idx_path)
        train_idx = pd.read_csv(train_idx_path, header=None)[0].values
        valid_idx = pd.read_csv(valid_idx_path, header=None)[0].values
    else:
        assert train_size, "please provide either train_size or none of sizes"
        if valid_size is None:
            valid_size = len(X_train) - train_size
            assert valid_size > 0
        if train_size + valid_size > len(X_train):
            warnings.warn('train_size + valid_size = {} exceeds dataset size: {}.'.format(
                train_size + valid_size, len(X_train)), Warning)
        if test_size is not None:
            warnings.warn('Test set is fixed for this dataset.', Warning)

        shuffled_indices = np.random.permutation(np.arange(len(X_train)))
        train_idx = shuffled_indices[:train_size]
        valid_idx = shuffled_indices[train_size: train_size + valid_size]

    return dict(
        X_train=X_train[train_idx], y_train=y_train[train_idx],
        X_valid=X_train[valid_idx], y_valid=y_train[valid_idx],
        X_test=X_test, y_test=y_test
    )


def fetch_PROTEIN(path, train_size=None, valid_size=None, test_size=None, **kwargs):
    """
    https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass.html#protein
    """
    train_path = os.path.join(path, 'protein')
    test_path = os.path.join(path, 'protein.t')
    if not all(os.path.exists(fname) for fname in (train_path, test_path)):
        os.makedirs(path, exist_ok=True)
        download("https://www.dropbox.com/s/pflp4vftdj3qzbj/protein.tr?dl=1", train_path)
        download("https://www.dropbox.com/s/z7i5n0xdcw57weh/protein.t?dl=1", test_path)
    for fname in (train_path, test_path):
        raw = open(fname).read().replace(' .', '0.')
        with open(fname, 'w') as f:
            f.write(raw)

    X_train, y_train = load_svmlight_file(train_path, dtype=np.float32, n_features=357)
    X_test, y_test = load_svmlight_file(test_path, dtype=np.float32, n_features=357)
    X_train, X_test = X_train.toarray(), X_test.toarray()
    y_train, y_test = y_train.astype(np.int), y_test.astype(np.int)

    if all(sizes is None for sizes in (train_size, valid_size, test_size)):
        train_idx_path = os.path.join(path, 'stratified_train_idx.txt')
        valid_idx_path = os.path.join(path, 'stratified_valid_idx.txt')
        if not all(os.path.exists(fname) for fname in (train_idx_path, valid_idx_path)):
            download("https://www.dropbox.com/s/wq2v9hl1wxfufs3/small_stratified_train_idx.txt?dl=1", train_idx_path)
            download("https://www.dropbox.com/s/7o9el8pp1bvyy22/small_stratified_valid_idx.txt?dl=1", valid_idx_path)
        train_idx = pd.read_csv(train_idx_path, header=None)[0].values
        valid_idx = pd.read_csv(valid_idx_path, header=None)[0].values
    else:
        assert train_size, "please provide either train_size or none of sizes"
        if valid_size is None:
            valid_size = len(X_train) - train_size
            assert valid_size > 0
        if train_size + valid_size > len(X_train):
            warnings.warn('train_size + valid_size = {} exceeds dataset size: {}.'.format(
                train_size + valid_size, len(X_train)), Warning)
        if test_size is not None:
            warnings.warn('Test set is fixed for this dataset.', Warning)

        shuffled_indices = np.random.permutation(np.arange(len(X_train)))
        train_idx = shuffled_indices[:train_size]
        valid_idx = shuffled_indices[train_size: train_size + valid_size]

    return dict(
        X_train=X_train[train_idx], y_train=y_train[train_idx],
        X_valid=X_train[valid_idx], y_valid=y_train[valid_idx],
        X_test=X_test, y_test=y_test
    )


def fetch_YEAR(path, train_size=None, valid_size=None, test_size=51630, **kwargs):
    data_path = os.path.join(path, 'data.csv')
    if not os.path.exists(data_path):
        os.makedirs(path, exist_ok=True)
        download('https://www.dropbox.com/s/l09pug0ywaqsy0e/YearPredictionMSD.txt?dl=1', data_path)
    n_features = 91
    types = {i: (np.float32 if i != 0 else np.int) for i in range(n_features)}
    data = pd.read_csv(data_path, header=None, dtype=types)
    data_train, data_test = data.iloc[:-test_size], data.iloc[-test_size:]

    X_train, y_train = data_train.iloc[:, 1:].values, data_train.iloc[:, 0].values
    X_test, y_test = data_test.iloc[:, 1:].values, data_test.iloc[:, 0].values

    if all(sizes is None for sizes in (train_size, valid_size)):
        train_idx_path = os.path.join(path, 'stratified_train_idx.txt')
        valid_idx_path = os.path.join(path, 'stratified_valid_idx.txt')
        if not all(os.path.exists(fname) for fname in (train_idx_path, valid_idx_path)):
            download("https://www.dropbox.com/s/00u6cnj9mthvzj1/stratified_train_idx.txt?dl=1", train_idx_path)
            download("https://www.dropbox.com/s/420uhjvjab1bt7k/stratified_valid_idx.txt?dl=1", valid_idx_path)
        train_idx = pd.read_csv(train_idx_path, header=None)[0].values
        valid_idx = pd.read_csv(valid_idx_path, header=None)[0].values
    else:
        assert train_size, "please provide either train_size or none of sizes"
        if valid_size is None:
            valid_size = len(X_train) - train_size
            assert valid_size > 0
        if train_size + valid_size > len(X_train):
            warnings.warn('train_size + valid_size = {} exceeds dataset size: {}.'.format(
                train_size + valid_size, len(X_train)), Warning)

        shuffled_indices = np.random.permutation(np.arange(len(X_train)))
        train_idx = shuffled_indices[:train_size]
        valid_idx = shuffled_indices[train_size: train_size + valid_size]

    return dict(
        X_train=X_train[train_idx], y_train=y_train[train_idx],
        X_valid=X_train[valid_idx], y_valid=y_train[valid_idx],
        X_test=X_test, y_test=y_test,
    )


def fetch_HIGGS(path, train_size=None, valid_size=None, test_size=5 * 10 ** 5, **kwargs):
    data_path = os.path.join(path, 'higgs.csv')
    if not os.path.exists(data_path):
        os.makedirs(path, exist_ok=True)
        archive_path = os.path.join(path, 'HIGGS.csv.gz')
        download('https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz', archive_path)
        with gzip.open(archive_path, 'rb') as f_in:
            with open(data_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    n_features = 29
    types = {i: (np.float32 if i != 0 else np.int) for i in range(n_features)}
    data = pd.read_csv(data_path, header=None, dtype=types)
    data_train, data_test = data.iloc[:-test_size], data.iloc[-test_size:]

    X_train, y_train = data_train.iloc[:, 1:].values, data_train.iloc[:, 0].values
    X_test, y_test = data_test.iloc[:, 1:].values, data_test.iloc[:, 0].values

    if all(sizes is None for sizes in (train_size, valid_size)):
        train_idx_path = os.path.join(path, 'stratified_train_idx.txt')
        valid_idx_path = os.path.join(path, 'stratified_valid_idx.txt')
        if not all(os.path.exists(fname) for fname in (train_idx_path, valid_idx_path)):
            download("https://www.dropbox.com/s/i2uekmwqnp9r4ix/stratified_train_idx.txt?dl=1", train_idx_path)
            download("https://www.dropbox.com/s/wkbk74orytmb2su/stratified_valid_idx.txt?dl=1", valid_idx_path)
        train_idx = pd.read_csv(train_idx_path, header=None)[0].values
        valid_idx = pd.read_csv(valid_idx_path, header=None)[0].values
    else:
        assert train_size, "please provide either train_size or none of sizes"
        if valid_size is None:
            valid_size = len(X_train) - train_size
            assert valid_size > 0
        if train_size + valid_size > len(X_train):
            warnings.warn('train_size + valid_size = {} exceeds dataset size: {}.'.format(
                train_size + valid_size, len(X_train)), Warning)

        shuffled_indices = np.random.permutation(np.arange(len(X_train)))
        train_idx = shuffled_indices[:train_size]
        valid_idx = shuffled_indices[train_size: train_size + valid_size]

    return dict(
        X_train=X_train[train_idx], y_train=y_train[train_idx],
        X_valid=X_train[valid_idx], y_valid=y_train[valid_idx],
        X_test=X_test, y_test=y_test,
    )


def fetch_MICROSOFT(path, **kwargs):
    train_path = os.path.join(path, 'msrank_train.tsv')
    test_path = os.path.join(path, 'msrank_test.tsv')
    if not all(os.path.exists(fname) for fname in (train_path, test_path)):
        os.makedirs(path, exist_ok=True)
        download("https://www.dropbox.com/s/izpty5feug57kqn/msrank_train.tsv?dl=1", train_path)
        download("https://www.dropbox.com/s/tlsmm9a6krv0215/msrank_test.tsv?dl=1", test_path)

        for fname in (train_path, test_path):
            raw = open(fname).read().replace('\\t', '\t')
            with open(fname, 'w') as f:
                f.write(raw)

    data_train = pd.read_csv(train_path, header=None, skiprows=1, sep='\t')
    data_test = pd.read_csv(test_path, header=None, skiprows=1, sep='\t')

    train_idx_path = os.path.join(path, 'train_idx.txt')
    valid_idx_path = os.path.join(path, 'valid_idx.txt')
    if not all(os.path.exists(fname) for fname in (train_idx_path, valid_idx_path)):
        download("https://www.dropbox.com/s/pba6dyibyogep46/train_idx.txt?dl=1", train_idx_path)
        download("https://www.dropbox.com/s/yednqu9edgdd2l1/valid_idx.txt?dl=1", valid_idx_path)
    train_idx = pd.read_csv(train_idx_path, header=None)[0].values
    valid_idx = pd.read_csv(valid_idx_path, header=None)[0].values

    X_train, y_train, query_train = data_train.iloc[train_idx, 2:].values, data_train.iloc[train_idx, 0].values, data_train.iloc[train_idx, 1].values
    X_valid, y_valid, query_valid = data_train.iloc[valid_idx, 2:].values, data_train.iloc[valid_idx, 0].values, data_train.iloc[valid_idx, 1].values
    X_test, y_test, query_test = data_test.iloc[:, 2:].values, data_test.iloc[:, 0].values, data_test.iloc[:, 1].values

    return dict(
        X_train=X_train.astype(np.float32), y_train=y_train.astype(np.int64), query_train=query_train,
        X_valid=X_valid.astype(np.float32), y_valid=y_valid.astype(np.int64), query_valid=query_valid,
        X_test=X_test.astype(np.float32), y_test=y_test.astype(np.int64), query_test=query_test,
    )


def fetch_YAHOO(path, *args):
    train_path = os.path.join(path, 'yahoo_train.tsv')
    valid_path = os.path.join(path, 'yahoo_valid.tsv')
    test_path = os.path.join(path, 'yahoo_test.tsv')
    if not all(os.path.exists(fname) for fname in (train_path, valid_path, test_path)):
        os.makedirs(path, exist_ok=True)
        train_archive_path = os.path.join(path, 'yahoo_train.tsv.gz')
        valid_archive_path = os.path.join(path, 'yahoo_valid.tsv.gz')
        test_archive_path = os.path.join(path, 'yahoo_test.tsv.gz')
        if not all(os.path.exists(fname) for fname in (train_archive_path, valid_archive_path, test_archive_path)):
            download("https://www.dropbox.com/s/7rq3ki5vtxm6gzx/yahoo_set_1_train.gz?dl=1", train_archive_path)
            download("https://www.dropbox.com/s/3ai8rxm1v0l5sd1/yahoo_set_1_validation.gz?dl=1", valid_archive_path)
            download("https://www.dropbox.com/s/3d7tdfb1an0b6i4/yahoo_set_1_test.gz?dl=1", test_archive_path)

        for file_name, archive_name in zip((train_path, valid_path, test_path), (train_archive_path, valid_archive_path, test_archive_path)):
            with gzip.open(archive_name, 'rb') as f_in:
                with open(file_name, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

        for fname in (train_path, valid_path, test_path):
            raw = open(fname).read().replace('\\t', '\t')
            with open(fname, 'w') as f:
                f.write(raw)

    data_train = pd.read_csv(train_path, header=None, skiprows=1, sep='\t')
    data_valid = pd.read_csv(valid_path, header=None, skiprows=1, sep='\t')
    data_test = pd.read_csv(test_path, header=None, skiprows=1, sep='\t')

    X_train, y_train, query_train = data_train.iloc[:, 2:].values, data_train.iloc[:, 0].values, data_train.iloc[:, 1].values
    X_valid, y_valid, query_valid = data_valid.iloc[:, 2:].values, data_valid.iloc[:, 0].values, data_valid.iloc[:, 1].values
    X_test, y_test, query_test = data_test.iloc[:, 2:].values, data_test.iloc[:, 0].values, data_test.iloc[:, 1].values

    return dict(
        X_train=X_train.astype(np.float32), y_train=y_train, query_train=query_train,
        X_valid=X_valid.astype(np.float32), y_valid=y_valid, query_valid=query_valid,
        X_test=X_test.astype(np.float32), y_test=y_test, query_test=query_test,
    )


def fetch_CLICK(path, valid_size=100_000, seed=None, **kwargs):
    # based on: https://www.kaggle.com/slamnz/primer-airlines-delay
    csv_path = os.path.join(path, 'click.csv')
    if not os.path.exists(csv_path):
        os.makedirs(path, exist_ok=True)
        download('https://www.dropbox.com/s/w43ylgrl331svqc/click.csv?dl=1', csv_path)

    data = pd.read_csv(csv_path, index_col=0)
    X, y = data.drop(columns=['target']), data['target']
    X_train, X_test = X[:-100_000].copy(), X[-100_000:].copy()
    y_train, y_test = y[:-100_000].copy(), y[-100_000:].copy()

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')

    cat_features = ['url_hash', 'ad_id', 'advertiser_id', 'query_id',
                    'keyword_id', 'title_id', 'description_id', 'user_id']

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=valid_size, random_state=seed)

    cat_encoder = LeaveOneOutEncoder()
    cat_encoder.fit(X_train[cat_features], y_train)
    X_train[cat_features] = cat_encoder.transform(X_train[cat_features])
    X_val[cat_features] = cat_encoder.transform(X_val[cat_features])
    X_test[cat_features] = cat_encoder.transform(X_test[cat_features])
    return dict(
        X_train=X_train.values.astype('float32'), y_train=y_train,
        X_valid=X_val.values.astype('float32'), y_valid=y_val,
        X_test=X_test.values.astype('float32'), y_test=y_test
    )

def fetch_MUSHROOMS(path, valid_size=0.2, test_size=0.2, seed=None):

    path = Path(path)
    data_path = path / 'agaricus-lepiota.data'

    if not data_path.exists():
        path.mkdir(parents=True, exist_ok=True)

        download('https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data', data_path)

    data = pd.read_csv(data_path, names=np.arange(23))
    encoder = OrdinalEncoder(return_df=False)
    data = encoder.fit_transform(data)
    
    X, Y = (data[:, 1:]).astype(np.float32), (data[:, 0] - 1).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=test_size, random_state=seed)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=valid_size / (1 - test_size), random_state=seed)

    return dict(
        X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test
    )

def fetch_TICTACTOE(path, valid_size=0.2, test_size=0.2, seed=None):

    path = Path(path)
    data_path = path / 'tic-tac-toe.data'

    if not data_path.exists():
        path.mkdir(parents=True, exist_ok=True)

        download('https://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data', data_path)

    data = pd.read_csv(data_path, names=np.arange(10))
    encoder = OrdinalEncoder(return_df=False)
    data = encoder.fit_transform(data)
    
    X, Y = (data[:, :-1]).astype(np.float32), (data[:, -1] - 1).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=test_size, random_state=seed)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=valid_size / (1 - test_size), random_state=seed)

    return dict(
        X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test
    )

#**from src.toy_datasets import ***

# ------------------------------------------------------------ TOY DATASETS

def toy_dataset(n=1000, distr="xor", dim=2):

    if distr == "xor":
        
        X = np.random.uniform(low=tuple([-1.] * dim), high=tuple([1.] * dim), size=(n, dim))
        Y = (X[:,0] * X[:,1] >= 0).astype(int)

        return dict(X=X.astype(np.float32), Y=Y)

    elif distr == "reg-xor":
        
        X = np.random.uniform(low=tuple([-1.] * dim), high=tuple([1.] * dim), size=(n, dim))
        labels = (X[:,0] * X[:,1] >= 0)

        Y = np.empty(n)
        Y[labels] = np.random.normal(0.8, 0.1, np.sum(labels))
        Y[~labels] = np.random.normal(0.2, 0.1, np.sum(~labels))

        return dict(X=X.astype(np.float32), Y=Y.astype(np.float32), labels=labels)

    elif distr == "swissroll":

        n2 = n // 2

        X1,_ = make_swiss_roll(n_samples=n2, noise=0)
        Y1 = np.ones(n2)

        X2 = np.random.uniform(low=tuple([-1.] * dim), high=tuple([1.] * dim), size=(n2, dim))
        Y2 = np.zeros(n2)

        X = np.r_[X1[:,::2] / 15, X2]
        Y = np.r_[Y1, Y2]

        return dict(X=X.astype(np.float32), Y=Y)

    else:
        NotImplementedError

**Preprocessing**

In [47]:
def preprocess_data_adult(data_path):
    # Read the data into a DataFrame
    columns = [
        "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
        "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"
    ]
    df = pd.read_csv(data_path, names=columns, na_values=[" ?"])

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Convert categorical features using Label Encoding
    categorical_columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Encode the target variable
    df["income"] = df["income"].apply(lambda x: 1 if x == " >50K" else 0)

    return df

def preprocess_data_bank_marketing(data):
    # Convert categorical features using Label Encoding
    label_encoders = {}
    for col in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

    return data

def preprocess_data_credit_card_defaults(data):
    # Convert categorical features using one-hot encoding
    data = pd.get_dummies(data, columns=["SEX", "EDUCATION", "MARRIAGE"], drop_first=True)

    # Standardize numerical features
    scaler = StandardScaler()
    data[["LIMIT_BAL", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1",
          "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2",
          "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]] = scaler.fit_transform(
        data[["LIMIT_BAL", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1",
               "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2",
               "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]])

    return data


**Fetch Tabular dataset**

In [48]:
def fetch_ADULT(data_dir="./ADULT_DATA"):
    print("---------------------ADULT--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        
    # URL of the dataset zip file
    url = "https://archive.ics.uci.edu/static/public/2/adult.zip"
    zip_file_path = os.path.join(data_dir, "adult.zip")
    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)
    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)

    # Preprocess the data
    train_data_path = os.path.join(data_dir, "adult.data")
#     test_data_path = os.path.join(data_dir, "adult.test")
   
    df_train = preprocess_data_adult(train_data_path)
#     df_test = preprocess_data_adult(test_data_path)

    # Split the data into train, validation, and test sets
    X = df_train.drop("income", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df_train["income"]
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
#     X_test = df_test.drop("income", axis=1)
#     y_test = df_test["income"]

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')
    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents using shutil.rmtree()
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train, X_valid=X_val.astype('float32'), y_valid=y_val, X_test=X_test.astype('float32'), y_test=y_test
    )

def fetch_bank_marketing(data_dir="./BANK"):
    print("---------------------BANK--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "https://archive.ics.uci.edu/static/public/222/bank+marketing.zip"
    zip_file_path = os.path.join(data_dir, "bank_marketing.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)
    
    zip_file_path_bank_add = os.path.join(data_dir, "bank-additional.zip")
    with zipfile.ZipFile(zip_file_path_bank_add, "r") as zip_ref:
        zip_ref.extractall(data_dir)

    # Get the extracted directory path
    extracted_dir = os.path.join(data_dir, "bank-additional")

    # Read the dataset
    data = pd.read_csv(os.path.join(extracted_dir, "bank-additional-full.csv"), sep=';')

    # Preprocess the data
    data = preprocess_data_bank_marketing(data)

    # Split the data into train, validation, and test sets
    X = data.drop("y", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = data["y"]
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')
    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,X_test=X_test.astype('float32'), y_test=y_test, X_valid = X_val.astype('float32'), y_valid = y_val
    )

def fetch_credit_card_defaults(data_dir="./CREDIT"):
    print("---------------------CREDIT--------------------------------------")
    # Create the data directory if it doesn't exist
    !pip install xlrd
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip"
    zip_file_path = os.path.join(data_dir, "credit_card_defaults.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)

#     # Get the extracted directory path
#     extracted_dir = os.path.join(data_dir, "default+of+credit+card+clients")

    # Read the dataset
    data = pd.read_excel(os.path.join(data_dir, "default of credit card clients.xls"), skiprows=1)

    # Preprocess the data
    data = preprocess_data_credit_card_defaults(data)

    # Split the data into train, validation, and test sets
    X = data.drop("default payment next month", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = data["default payment next month"]
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train, X_valid=X_val.astype('float32'), y_valid=y_val , X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_gamma_telescope(data_dir="./TELESCOPE"):
    print("---------------------TELESCOPE--------------------------------------")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "https://archive.ics.uci.edu/static/public/159/magic+gamma+telescope.zip"
    zip_file_path = os.path.join(data_dir, "magic_gamma_telescope.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)
    
    # Load the data from CSV
    data_path = os.path.join(data_dir, "magic04.data")
    columns = [
        "fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long",
        "fM3Trans", "fAlpha", "fDist", "class"
    ]
    data = pd.read_csv(data_path, header=None, names=columns)
    
    # Convert the class labels to binary format (g = gamma, h = hadron)
    data["class"] = data["class"].map({"g": 1, "h": 0})
    
    # Split the data into features (X) and target (y)
    X = data.drop("class", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = data["class"]
    
    # Split the data into train, test, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_valid = (y_valid.values.reshape(-1) == 1).astype('int64')
    
    # Create a dictionary to store the data splits
    data_splits = {
        "X_train": X_train.astype('float32'), "y_train": y_train,
        "X_valid": X_valid.astype('float32'), "y_valid": y_valid,
        "X_test": X_test.astype('float32'), "y_test": y_test
    }
    
    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir)
    
    return data_splits

def fetch_rice_dataset(data_dir="./RICE"):
    print("---------------------RICE--------------------------------------")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "https://archive.ics.uci.edu/static/public/545/rice+cammeo+and+osmancik.zip"
    zip_file_path = os.path.join(data_dir, "rice_dataset.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)
        
    # Load the data from CSV
    arff_file_name = os.path.join(data_dir, "Rice_Cammeo_Osmancik.arff")

    
    # Load the ARFF file using SciPy
    data, meta = arff.loadarff(arff_file_name)
    
    df = pd.DataFrame(data)
    print("df",df)
    df["Class"] = df["Class"].map({b'Cammeo': 1, b'Osmancik': 0})
    
    # Split the data into features (X) and target (y)
    X = df.drop("Class", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df["Class"]
    
    # Split the data into train, test, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_valid = (y_valid.values.reshape(-1) == 1).astype('int64')
    
    # Create a dictionary to store the data splits
    data_splits = {
        "X_train": X_train.astype('float32'), "y_train": y_train,
        "X_valid": X_valid.astype('float32'), "y_valid": y_valid,
        "X_test": X_test.astype('float32'), "y_test": y_test
    }
    
    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir)
    
    return data_splits

def fetch_german_credit_data(data_dir="./GERMAN"):
    print("---------------------GERMAN--------------------------------------")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "http://archive.ics.uci.edu/static/public/144/statlog+german+credit+data.zip"
    zip_file_path = os.path.join(data_dir, "german_credit_data.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)
        
    # Load the data from CSV
    data_path = os.path.join(data_dir, "german.data")

    columns = [
        "checking_account_status", "duration_months", "credit_history", "purpose",
        "credit_amount", "savings_account_bonds", "employment", "installment_rate",
        "personal_status_sex", "other_debtors_guarantors", "present_residence",
        "property", "age", "other_installment_plans", "housing", "existing_credits",
        "job", "num_dependents", "own_telephone", "foreign_worker", "class"
    ]
    data = pd.read_csv(data_path, sep=' ', header=None, names=columns)
    
    # Convert the class labels to binary format (1 = Good, 2 = Bad)
    data["class"] = data["class"].map({1: 1, 2: 0})
    
    # Handle null values (replace with appropriate values)
    data.fillna(method='ffill', inplace=True)  # Forward fill
    
    # Convert categorical variables to dummy variables
    categorical_columns = [
        "checking_account_status", "credit_history", "purpose", "savings_account_bonds",
        "employment", "personal_status_sex", "other_debtors_guarantors", "property",
        "other_installment_plans", "housing", "job", "own_telephone", "foreign_worker"
    ]
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    
    # Split the data into features (X) and target (y)
    X = data.drop("class", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = data["class"]
    
    # Split the data into train, test, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_valid = (y_valid.values.reshape(-1) == 1).astype('int64')
    
    # Create a dictionary to store the data splits
    data_splits = {
        "X_train": X_train.astype('float32'), "y_train": y_train,
        "X_valid": X_valid.astype('float32'), "y_valid": y_valid,
        "X_test": X_test.astype('float32'), "y_test": y_test
    }
    
    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir)
    
    return data_splits

def fetch_spambase_dataset(data_dir="./SPAM"):
    print("---------------------SPAM--------------------------------------")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "http://archive.ics.uci.edu/static/public/94/spambase.zip"
    zip_file_path = os.path.join(data_dir, "spambase.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)
        
    # Load the data from CSV
    data_path = os.path.join(data_dir, "spambase.data")

    columns = [
        f"f{i}" for i in range(57)
    ] + ["spam"]
    data = pd.read_csv(data_path, header=None, names=columns)
    
    # Split the data into features (X) and target (y)
    X = data.drop("spam", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = data["spam"]
    
    # Split the data into train, test, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_valid = (y_valid.values.reshape(-1) == 1).astype('int64')
    
    # Create a dictionary to store the data splits
    data_splits = {
        "X_train": X_train.astype('float32'), "y_train": y_train,
        "X_valid": X_valid.astype('float32'), "y_valid": y_valid,
        "X_test": X_test.astype('float32'), "y_test": y_test
    }
    
    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir)
    
    return data_splits

def fetch_accelerometer_gyro_dataset(data_dir="./GYRO"):
    print("---------------------GYRO--------------------------------------")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "https://archive.ics.uci.edu/static/public/755/accelerometer+gyro+mobile+phone+dataset.zip"
    zip_file_path = os.path.join(data_dir, "accelerometer_gyro_dataset.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)
        
    # Load the data from CSV
    data_path = os.path.join(data_dir, "accelerometer_gyro_mobile_phone_dataset.csv")
    
    data = pd.read_csv(data_path)
    
    # Convert categorical column to numeric (e.g., label encoding)
    data["timestamp"] = data["timestamp"].astype("category").cat.codes
    
    # Split the data into features (X) and target (y)
    X = data.drop("Activity", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = data["Activity"]
    
    # Split the data into train, test, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_valid = (y_valid.values.reshape(-1) == 1).astype('int64')
    
    # Create a dictionary to store the data splits
    data_splits = {
        "X_train": X_train.astype('float32'), "y_train": y_train,
        "X_valid": X_valid.astype('float32'), "y_valid": y_valid,
        "X_test": X_test.astype('float32'), "y_test": y_test
    }
    
    # Remove the zip file
    os.remove(zip_file_path)

    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir)
    
    return data_splits

def fetch_swarm_behaviour(data_dir="./SWARM"):
    print("---------------------SWARM--------------------------------------")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # URL of the dataset zip file
    url = "https://archive.ics.uci.edu/static/public/524/swarm+behaviour.zip"
    zip_file_path = os.path.join(data_dir, "swarm_behaviour.zip")

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)
        
    # Load the data from CSV
    data_path = os.path.join(data_dir, "Swarm Behavior Data/Grouped.csv")
    
    data = pd.read_csv(data_path)
    
    # Split the data into features (X) and target (y)
    X = data.drop("Class", axis=1)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = data["Class"]
    
    # Split the data into train, test, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_valid = (y_valid.values.reshape(-1) == 1).astype('int64')
    
    # Create a dictionary to store the data splits
    data_splits = {
        "X_train": X_train.astype('float32'), "y_train": y_train,
        "X_valid": X_valid.astype('float32'), "y_valid": y_valid,
        "X_test": X_test.astype('float32'), "y_test": y_test
    }
    
    # Remove the zip file
    os.remove(zip_file_path)
    # Remove the extracted directory and its contents
    shutil.rmtree(data_dir) 
    return data_splits

def fetch_openml_credit_data(data_dir="./OpenML_Credit"):
    print("---------------------OpenML_Credit DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22103185/credit.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "credit.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]

    df[last_column] = df[last_column].astype(int)
    
#     print("df",df)

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_electricity_data(data_dir="./OpenML_Electricity"):
    print("---------------------OpenML_Electricity DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22103245/electricity.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "electricity.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]

    df[last_column] = df[last_column].map({b'DOWN': 0, b'UP': 1})
    
#     print("df",df)

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_covertype_data(data_dir="./OpenML_Covertype"):
    print("---------------------OpenML_Covertype DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22103246/covertype.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "covertype.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]

    df[last_column] = df[last_column].astype(int)
    
#     print("df",df)

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_pol_data(data_dir="./OpenML_Pol"):
    print("---------------------OpenML_Pol DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22103247/pol.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "pol.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]

#     print("df",df)
    
    df[last_column] = df[last_column].map({b'N':0,b'P':1})
    
    

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_house_16H_data(data_dir="./OpenML_House_16H"):
    print("---------------------OpenML_House_16H DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22103248/house_16H.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "house_16H.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]

#     print("df",df)
    df[last_column] = df[last_column].map({b'N':0,b'P':1})
    
    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_MiniBooNE_data(data_dir="./OpenML_MiniBooNE"):
    print("---------------------OpenML_MiniBooNE DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22103253/MiniBooNE.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "MiniBooNE.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]

#     print("df",df)
    
    df[last_column] = df[last_column].map({b'False':0,b'True':1})

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_eye_movements_data(data_dir="./OpenML_Eye_movements"):
    print("---------------------OpenML_Eye_movements DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22103255/eye_movements.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "eye_movements.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]

#     print("df",df)
    df[last_column] = df[last_column].astype(int)

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_Diabetes130US_data(data_dir="./OpenML_Diabetes130US"):
    print("---------------------OpenML_Diabetes130US DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22111908/Diabetes130US.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "Diabetes130US.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]
#     print("df",df)
    df[last_column] = df[last_column].astype(int)
    

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_jannis_data(data_dir="./OpenML_Jannis"):
    print("---------------------OpenML_Jannis DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22111907/jannis.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "jannis.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]
#     print("df",df)

    df[last_column] = df[last_column].astype(int)


    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_Bioresponse_data(data_dir="./OpenML_Bioresponse"):
    print("---------------------OpenML_Bioresponse DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22111905/Bioresponse.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "Bioresponse.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]
#     print("df",df)

    df[last_column] = df[last_column].astype(int)

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_california_data(data_dir="./OpenML_California"):
    print("---------------------OpenML_California DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22111914/california.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "california.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]
#     print("df",df)

    df[last_column] = df[last_column].astype(int)

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


def fetch_openml_heloc_data(data_dir="./OpenML_Heloc"):
    print("---------------------OpenML_Heloc DATASET--------------------------------------")
    # Create the data directory if it doesn't exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    data_url = "https://api.openml.org/data/v1/download/22111912/heloc.arff"
    # Download the ARFF file
    arff_file_path = os.path.join(data_dir, "heloc.arff")
    urllib.request.urlretrieve(data_url, arff_file_path)

    # Load ARFF file into DataFrame
    data, meta = arff.loadarff(arff_file_path)
    df = pd.DataFrame(data)
    # Convert target variable to int
    last_column = df.columns[-1]
#     print("df",df)

    df[last_column] = df[last_column].astype(int)

    # Split the data into train, validation, and test sets
    X = df.drop(last_column, axis=1)  # Assuming "SeriousDlqin2yrs" is the target variable
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = df[last_column]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

#     y_train = y_train.astype('int64')
#     y_test = y_test.astype('int64')
#     y_val = y_val.astype('int64')

    y_train = (y_train.values.reshape(-1) == 1).astype('int64')
    y_test = (y_test.values.reshape(-1) == 1).astype('int64')
    y_val = (y_val.values.reshape(-1) == 1).astype('int64')

    # Remove the ARFF file
    os.remove(arff_file_path)

    # Remove the data directory
    shutil.rmtree(data_dir)

    return dict(
        X_train=X_train.astype('float32'), y_train=y_train,
        X_valid=X_val.astype('float32'), y_valid=y_val,
        X_test=X_test.astype('float32'), y_test=y_test
    )


In [49]:
#**class Dataset:**

REAL_DATASETS = {
    'A9A': fetch_A9A,
    'EPSILON': fetch_EPSILON,
    'PROTEIN': fetch_PROTEIN,
    'YEAR': fetch_YEAR,
    'MICROSOFT': fetch_MICROSOFT,
    'YAHOO': fetch_YAHOO,
    'CLICK': fetch_CLICK,
    'GLASS': fetch_GLASS,
    'COVTYPE': fetch_COVTYPE,
    'ALOI': fetch_ALOI,
    'DIGITS': fetch_DIGITS,
    'MUSH': fetch_MUSHROOMS,
    'TTT': fetch_TICTACTOE,
    ####### 10 latest UCI datasets ########
    'ADULT': fetch_ADULT,
    'bank_marketing': fetch_bank_marketing,
    'credit_card_defaults': fetch_credit_card_defaults,
    'gamma_telescope': fetch_gamma_telescope,
    'rice_dataset': fetch_rice_dataset,
    'german_credit_data': fetch_german_credit_data,
    'spambase_dataset': fetch_spambase_dataset,
    'accelerometer_gyro_dataset': fetch_accelerometer_gyro_dataset,
    'swarm_behaviour': fetch_swarm_behaviour,
    'HIGGS': fetch_HIGGS,
    ######## OpenML Tabular Datasets ##########
    'OpenML_Credit': fetch_openml_credit_data,
    'OpenML_Electricity': fetch_openml_electricity_data,
    'OpenML_Covertype': fetch_openml_covertype_data,
    'OpenML_Pol': fetch_openml_pol_data,
    'OpenML_House_16H': fetch_openml_house_16H_data,
    'OpenML_MiniBooNE': fetch_openml_MiniBooNE_data,
    'OpenML_Eye_movements': fetch_openml_eye_movements_data,
    'OpenML_Diabetes130US': fetch_openml_Diabetes130US_data,
    'OpenML_Jannis': fetch_openml_jannis_data,
    'OpenML_Bioresponse': fetch_openml_Bioresponse_data,
    'OpenML_California': fetch_openml_california_data,
    'OpenML_Heloc': fetch_openml_heloc_data
}

TOY_DATASETS = [
    'xor',
    'reg-xor',
    'swissroll',
]

class Dataset:
    def __init__(self, dataset, data_path='./DATA', normalize=False, normalize_target=False, quantile_transform=False, quantile_noise=1e-3, in_features=None, out_features=None, flatten=False, **kwargs):
        """
        Dataset is a dataclass that contains all training and evaluation data required for an experiment
        :param dataset: a pre-defined dataset name (see DATASETS) or a custom dataset
            Your dataset should be at (or will be downloaded into) {data_path}/{dataset}
        :param data_path: a shared data folder path where the dataset is stored (or will be downloaded into)
        :param normalize: standardize features by removing the mean and scaling to unit variance
        :param quantile_transform: whether tranform the feature distributions into normals, using a quantile transform
        :param quantile_noise: magnitude of the quantile noise
        :param in_features: which features to use as inputs
        :param out_features: which features to reconstruct as output
        :param flatten: whether flattening instances to vectors
        :param kwargs: depending on the dataset, you may select train size, test size or other params
        """

        if dataset in REAL_DATASETS:
            data_dict = REAL_DATASETS[dataset](Path(data_path) / dataset, **kwargs)

            self.X_train = data_dict['X_train']
            self.y_train = data_dict['y_train']
            self.X_valid = data_dict['X_valid']
            self.y_valid = data_dict['y_valid']
            self.X_test = data_dict['X_test']
            self.y_test = data_dict['y_test']

            if flatten:
                self.X_train, self.X_valid, self.X_test = self.X_train.reshape(len(self.X_train), -1), self.X_valid.reshape(len(self.X_valid), -1), self.X_test.reshape(len(self.X_test), -1)

            if normalize:

                print("Normalize dataset")
                axis = [0] + [i + 2 for i in range(self.X_train.ndim - 2)]
                self.mean = np.mean(self.X_train, axis=tuple(axis), dtype=np.float32)
                self.std = np.std(self.X_train, axis=tuple(axis), dtype=np.float32)

                # if constants, set std to 1
                self.std[self.std == 0.] = 1.

                if dataset not in ['ALOI']:
                    self.X_train = (self.X_train - self.mean) / self.std
                    self.X_valid = (self.X_valid - self.mean) / self.std
                    self.X_test = (self.X_test - self.mean) / self.std

            if quantile_transform:
                quantile_train = np.copy(self.X_train)
                if quantile_noise:
                    stds = np.std(quantile_train, axis=0, keepdims=True)
                    noise_std = quantile_noise / np.maximum(stds, quantile_noise)
                    quantile_train += noise_std * np.random.randn(*quantile_train.shape)

                qt = QuantileTransformer(output_distribution='normal').fit(quantile_train)
                self.X_train = qt.transform(self.X_train)
                self.X_valid = qt.transform(self.X_valid)
                self.X_test = qt.transform(self.X_test)

            if normalize_target:

                print("Normalize target value")
                self.mean_y = np.mean(self.y_train, axis=0, dtype=np.float32)
                self.std_y = np.std(self.y_train, axis=0, dtype=np.float32)

                # if constants, set std to 1
                if self.std_y == 0.:
                    self.std_y = 1.

                self.y_train = (self.y_train - self.mean_y) / self.std_y
                self.y_valid = (self.y_valid - self.mean_y) / self.std_y
                self.y_test = (self.y_test - self.mean_y) / self.std_y

            if in_features is not None:
                self.X_train_in, self.X_valid_in, self.X_test_in = self.X_train[:, in_features], self.X_valid[:, in_features], self.X_test[:, in_features]

            if out_features is not None:
                self.X_train_out, self.X_valid_out, self.X_test_out = self.X_train[:, out_features], self.X_valid[:, out_features], self.X_test[:, out_features]

        elif dataset in TOY_DATASETS:
            data_dict = toy_dataset(distr=dataset, **kwargs)

            self.X = data_dict['X']
            self.Y = data_dict['Y']
            if 'labels' in data_dict:
                self.labels = data_dict['labels']

        self.data_path = data_path
        self.dataset = dataset

class TorchDataset(torch.utils.data.Dataset):

    def __init__(self, *data, **options):
        
        n_data = len(data)
        if n_data == 0:
            raise ValueError("At least one set required as input")

        self.data = data
        means = options.pop('means', None)
        stds = options.pop('stds', None)
        self.transform = options.pop('transform', None)
        self.test = options.pop('test', False)
        
        if options:
            raise TypeError("Invalid parameters passed: %s" % str(options))
         
        if means is not None:
            assert stds is not None, "must specify both <means> and <stds>"

            self.normalize = lambda data: [(d - m) / s for d, m, s in zip(data, means, stds)]

        else:
            self.normalize = lambda data: data

    def __len__(self):
        return len(self.data[0])

    def __getitem__(self, idx):
        data = self.normalize([s[idx] for s in self.data])
        if self.transform:

            if self.test:
                data = sum([[self.transform.test_transform(d)] * 2 for d in data], [])
            else:
                data = sum([self.transform(d) for d in data], [])
            
        return data

**DLGN Model**

In [50]:
def set_npseed(seed):
    np.random.seed(seed)


def set_torchseed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
class DLGN_FC(nn.Module):
    def __init__(self, input_dim=None, output_dim=None, num_hidden_nodes=[], beta=30, dlgn_mode='dlgn_sf', mode='pwc'):
        super(DLGN_FC, self).__init__()
        self.num_hidden_layers = len(num_hidden_nodes)
        self.beta=beta  # Soft gating parameter
        self.dlgn_mode = dlgn_mode
        self.mode = mode
        self.num_nodes=[input_dim]+num_hidden_nodes+[output_dim]
        self.gating_layers=nn.ModuleList()
        self.value_layers=nn.ModuleList()

        for i in range(self.num_hidden_layers+1):
            if i!=self.num_hidden_layers:
                if self.dlgn_mode == 'dlgn_sf':
                    temp = nn.Linear(self.num_nodes[0], self.num_nodes[i+1], bias=False)
                else :
                    temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
                self.gating_layers.append(temp)
            temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
            self.value_layers.append(temp)
    
    def set_parameters_with_mask(self, to_copy, parameter_masks):
        # self and to_copy are DLGN_FC objects with same architecture
        # parameter_masks is compatible with dict(to_copy.named_parameters())
        for (name, copy_param) in to_copy.named_parameters():
            copy_param = copy_param.clone().detach()
            orig_param  = self.state_dict()[name]
            if name in parameter_masks:
                param_mask = parameter_masks[name]>0
                orig_param[param_mask] = copy_param[param_mask]
            else:
                orig_param = copy_param.data.detach()

    def return_gating_functions(self):
        effective_weights = []
        for i in range(self.num_hidden_layers):
            curr_weight = self.gating_layers[i].weight.detach().clone()
            curr_weight /= torch.norm(curr_weight, dim=1, keepdim=True)
            if self.dlgn_mode=='dlgn_sf':
                effective_weights.append(curr_weight)
            else:
                if i==0:
                    effective_weights.append(curr_weight)
                else:
                    effective_weights.append(torch.matmul(curr_weight,effective_weights[-1]))
        return effective_weights
        # effective_weights (and effective biases) is a list of size num_hidden_layers


    def forward(self, x):
        gate_scores=[x]

        for el in self.parameters():
            if el.is_cuda:
                device = torch.device('cuda:0')
                # device = torch.device('cpu')
            else:
                device = torch.device('cpu')
        if self.mode=='pwc':
            values=[torch.ones(x.shape).to(device)]
        else:
            values=[x.to(device)]

        for i in range(self.num_hidden_layers):
            if self.dlgn_mode=='dlgn_sf':
                gate_scores.append( (x@self.gating_layers[i].weight.T) )
            else:
                gate_scores.append(self.gating_layers[i].to(device)(gate_scores[-1].to(device)))
            curr_gate_on_off = torch.sigmoid(self.beta * gate_scores[-1])
            values.append(self.value_layers[i](values[-1])*curr_gate_on_off)
        values.append(self.value_layers[self.num_hidden_layers](values[-1]))
        # Values is a list of size 1+num_hidden_layers+1
        #gate_scores is a list of size 1+num_hidden_layers
        return values,gate_scores

In [51]:
#from src.optimization import train_stochastic, evaluate

def train_stochastic(dataloader, model, optimizer, criterion,criterion2, epoch,parameter_mask=dict(), monitor=None, prog_bar=True):

    model.train()

    last_iter = epoch * len(dataloader)
#     print("last_iter inside train_stochastic:",last_iter)
    train_obj = 0.
    total_losses=0.
    if prog_bar:
        pbar = tqdm(dataloader)
    else:
        pbar = dataloader

    for i, batch in enumerate(pbar):
        optimizer.zero_grad()        

        t_x, t_y = batch
        t_x = t_x.to(device)
        t_y = t_y.to(device)
        if(len(t_x)>1):
            if t_y.dim() > 2: # predictors support only flatten output atm
                t_y = t_y.view(len(t_y), -1).to(device)

            values,gate_scores = model(t_x)
            y_pred = torch.sigmoid(values[-1])
            y_pred = y_pred.squeeze()
            loss = criterion(y_pred, t_y) / len(t_x)
            train_obj += loss.cpu().detach().numpy()

            ##Testing 0-1 loss
            y_pred1=y_pred.clone()
            y_pred1[y_pred > 0.5] = 1
            y_pred1[y_pred <= 0.5] = 0
            y_pred1 = y_pred1.detach()
            loss2 = criterion2(y_pred1, t_y)/ len(t_x)
            total_losses += loss2.cpu().detach()
            ##Testing 0-1 loss


            if prog_bar:
                pbar.set_description("avg train loss %f" % (total_losses / (i + 1)))
            loss.backward()
            for name,param in model.named_parameters():
                parameter_mask[name] = parameter_mask[name].to(device)
                param.grad *= parameter_mask[name]
#             for name,param in model.named_parameters():
#                 param.grad *= parameter_mask[name].to(device)

            optimizer.step()

            if monitor:
                monitor.write(model, i + last_iter, report_tree=True, train={"Loss": loss.cpu().detach()})
            
def evaluate(dataloader, model, criteria, epoch=None, monitor=None):

    model.eval()
    total_losses = {k: 0. for k in criteria.keys()}
    num_points = 0
    for batch in dataloader:

        t_x, t_y = batch
        t_x = t_x.to(device)
        t_y = t_y.to(device)
        
        if t_y.dim() > 2: # predictors support only flatten output atm
            t_y = t_y.view(len(t_y), -1)
        
        num_points += len(t_x)
        
        values,gate_scores = model(t_x)
        
        y_pred = torch.sigmoid(values[-1])
        y_pred = y_pred.squeeze()

        y_pred[y_pred > 0.5] = 1
        y_pred[y_pred <= 0.5] = 0
        y_pred = y_pred.detach()

        for k in criteria.keys():
            loss = criteria[k](y_pred, t_y)
            total_losses[k] += loss.cpu().detach()

    if monitor:
        monitor.write(model, epoch, val={k: loss / num_points for k, loss in total_losses.items()})

    return {k: loss.cpu().numpy() / num_points for k, loss in total_losses.items()}

In [52]:
def train_dlgn_classification (DLGN_obj,trainloader,valloader,num_epoch=1,parameter_mask=dict()):
    # DLGN_obj is the initial network
    # parameter_mask is a dictionary compatible with dict(DLGN_obj.named_parameters())
    # if a key corresponding to a named_parameter is not present it is assumed to be all ones (i.e it will be updated)
    
    set_torchseed(seed)
    DLGN_obj.to(device)
#     DLGN_obj_initial = DLGN_FC(to_copy=DLGN_obj)
#     DLGN_obj_return = DLGN_FC(to_copy=DLGN_obj)
    
    
    # init loss
    loss = nn.BCELoss(reduction="sum")
    criterion = lambda x, y: loss(x.float(), y.float())
    # evaluation criterion => error rate
    eval_criterion = lambda x, y: (x != y).sum()
       
    
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(DLGN_obj.parameters(), lr=lr)
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(DLGN_obj.parameters(), lr=lr)
        
        
#     # init optimizer
#     optimizer = QHAdam(model.parameters(), lr=LR, nus=(0.7, 1.0), betas=(0.995, 0.998))
    
    
    losses=[]
    DLGN_obj_store = []
    test_losses, train_times, test_times = [], [], []
    

    best_val_loss = float("inf")
    best_e = -1
    no_improv = 0
    t0 = time.time()
    for e in range(num_epoch):
        train_stochastic(trainloader, DLGN_obj, optimizer, criterion,eval_criterion, epoch=e,parameter_mask=parameter_mask, monitor=None,prog_bar=False)

        val_loss = evaluate(valloader, DLGN_obj, {'ER': eval_criterion}, epoch=e, monitor=None)
#         print("Epoch %i: validation loss = %f\n" % (e, val_loss["ER"]))
        no_improv += 1
        if val_loss["ER"] < best_val_loss:
            best_val_loss = val_loss["ER"]
            best_e = e
            no_improv = 0
#             if perfectSq(epoch):
#             DLGN_obj_store.append(DLGN_FC(to_copy=DLGN_obj))
            DLGN_obj_return = deepcopy(DLGN_obj)
#             LTBinaryClassifier.save_model(model, optimizer, state, save_dir, epoch=e, val_er=best_val_loss)

        if no_improv == num_epoch // 5:
            break
    t1 = time.time()

    print("best validation error rate (epoch {}): {}\n".format(best_e, best_val_loss))

    
    model = DLGN_obj_return
    t2 = time.time()
    test_loss = evaluate(testloader, model, {'ER': eval_criterion})
    print("test error rate (model of epoch {}): {}\n".format(best_e, test_loss['ER']))
    print("test accuracy (model of epoch {}): {}\n".format(best_e,1-test_loss['ER']))

#     t3 = time.time()
#     test_losses.append(test_loss['ER'])
#     train_times.append(t1 - t0)
#     test_times.append(t3 - t2)

#     print(np.mean(test_losses), np.std(test_losses))
# #     np.save(save_dir / '../test-losses.npy', test_losses)
#     print("Avg train time", np.mean(train_times))
#     print("Avg test time", np.mean(test_times))
    
    return DLGN_obj_return, DLGN_obj_store

**Change this cell to run different configs**

In [53]:
weight_decay=0.0
num_hidden_layers=3

output_dim=1 #num_of_classes
# num_hidden_nodes=[50,50,50]
modep='pwc'
dlgn_mode='dlgn' #use dlgn_sf for DLGN-SF model
model_name = 'DLGN' #DLGN/DeepnonLinearModel
beta = 30
optimizer_name ='Adam'

seed = 365
lr = 0.001
BATCH_SIZE = 512 

num_hidden_nodes_list=[[100,100,100,100]]#,[20,20,20],[50,50,50],[100,100,100],[10,10,10,10],[20,20,20,20],[50,50,50,50],[100,100,100,100],[10,10,10,10,10],[20,20,20,20],[50,50,50,50,50],[100,100,100,100,100]]
lr_list = [0.001]
num_epoch = 500
# DATA_NAME=["OpenML_Credit","OpenML_Electricity","OpenML_Covertype","OpenML_Pol","OpenML_House_16H","OpenML_MiniBooNE","OpenML_Eye_movements","OpenML_Diabetes130US","OpenML_Jannis","OpenML_Bioresponse","OpenML_California","OpenML_Heloc"]#"OpenML_Credit"]#,"bank_marketing","credit_card_defaults","gamma_telescope","rice_dataset","german_credit_data","spambase_dataset","accelerometer_gyro_dataset","swarm_behaviour"]#,"HIGGS"]
DATA_NAME=["bank_marketing"]
for data_name in DATA_NAME:
    for num_hidden_nodes in num_hidden_nodes_list:
        num_hidden_layers = len(num_hidden_nodes)
        for lr in lr_list:
            print("LR Valeu:",lr)
            print("=================",num_hidden_layers,num_hidden_nodes,"=========================")
            data = Dataset(data_name,normalize=True)
            print('classes', np.unique(data.y_test))
    #         print("data",data.y_train)
            set_npseed(seed)
            set_torchseed(seed)

            save_dir = Path("./results/tabular-quantile/") / data_name / "seed={}".format(seed)
            save_dir.mkdir(parents=True, exist_ok=True)

            input_dim=data.X_train.shape[1]
            trainloader = torch.utils.data.DataLoader(TorchDataset(data.X_train, data.y_train), batch_size=BATCH_SIZE, shuffle=True)
            valloader = torch.utils.data.DataLoader(TorchDataset(data.X_valid, data.y_valid), batch_size=BATCH_SIZE, shuffle=False)
            testloader = torch.utils.data.DataLoader(TorchDataset(data.X_test, data.y_test), batch_size=BATCH_SIZE, shuffle=False)

#             set_torchseed(6675)
            DLGN_init= DLGN_FC(input_dim=input_dim, output_dim=1, num_hidden_nodes=num_hidden_nodes, beta=beta, dlgn_mode=dlgn_mode)
#             for name,parameter in DLGN_init.named_parameters():
#                 print(name)
#                 print(parameter.shape)
            train_parameter_masks=dict()
            for name,parameter in DLGN_init.named_parameters():
                if "val" in name:
                    train_parameter_masks[name]=torch.ones_like(parameter) #*0.001 # Updating all value network layers
                if "gat" in name:
                    train_parameter_masks[name]=torch.ones_like(parameter)


                    # train_parameter_masks[name][:num_neurons_set] *= 0.
                train_parameter_masks[name].to(device)
            
#             set_torchseed(5000)
            DLGN_obj_return,DLGN_obj_store = train_dlgn_classification(DLGN_obj=deepcopy(DLGN_init),trainloader=trainloader,valloader=valloader,num_epoch=num_epoch,parameter_mask=train_parameter_masks)

            torch.cuda.empty_cache() 
#             losses=np.array(losses)

            ## For single run


        '''

        ##For multiple runs

        lr_store = [0.001,0.0001,0.00001]
        num_hidden_nodes_store=[[2,2,2],[5,5,5],[10,10,10],[20,20,20],[50,50,50],[100,100,100],[200,200,200],[500,500,500]]

        for lr in lr_store:
            for num_hidden_nodes in num_hidden_nodes_store:
                print("=========================================================================================")
                print("LR Valeu:",lr)
                print("num_hidden_nodes:",num_hidden_nodes)
                print("=========================================================================================")
                lr=lr
                num_hidden_nodes=num_hidden_nodes
                DLGN_init= DLGN_FC(input_dim=input_dim, output_dim=1, num_hidden_nodes=num_hidden_nodes,beta=beta)
                DLGN_obj_return,DLGN_obj_store=train_model_func(DLGN_init,trainloader,valloader,num_epoch,NPF_freeze,NPV_freeze)
                del DLGN_init
                del DLGN_obj_store

        '''

LR Valeu: 0.001
================= 4 [100, 100, 100, 100] =========================
---------------------BANK--------------------------------------
Normalize dataset
classes [0 1]
best validation error rate (epoch 8): 0.08497208060208788

test error rate (model of epoch 8): 0.08642874484098081

test accuracy (model of epoch 8): 0.9135712551590192

